In [94]:
    # # window_size_ma=20
    # threshold_param=100#no use
    # long_probability_threshold=0.70
    # short_probability_threshold=0.70
    # fees = 0.0015 
    # stop_loss = 0.01
    # sl_trail = 0.02
    # position_sizes = {
    #     20: 0.02,  
    #     50: 0.25,  
    #     70: 0.85   
    # }

In [125]:
window_size_ma=20
threshold_param=100#no use
long_probability_threshold=0.65
short_probability_threshold=0.65
fees = 0.0015
stop_loss = 0.02
# take_profit = 0.0
sl_trail = 0.01
position_sizes = {
    20: 0.04,  
    50: 0.55,  
    70: 0.85   
}

In [126]:
import pandas as pd
import vectorbt as vbt

data = pd.read_csv('btc_1h_test.csv')
column_names = ['datetime', 'open', 'high', 'low', 'close', 'volume']
data.columns = column_names

data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime', inplace=True)
data

,open,high,low,close,volume
datetime,,,,,
2021-02-01 01:30:00,32807.29,32931.55,32461.23,32853.73,2968.315197
2021-02-01 02:30:00,32853.73,32899.39,32515.00,32560.73,2336.553165
2021-02-01 03:30:00,32561.35,33088.88,32559.85,32974.10,1874.807965
2021-02-01 04:30:00,32974.10,33255.49,32933.54,33092.98,3218.738903
2021-02-01 05:30:00,33092.97,33106.33,32296.16,32546.27,4383.926122
...,...,...,...,...,...
2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.922560
2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.060520
2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.658550


In [127]:
nan_values = data.isna().sum()
nan_values

open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [128]:
rsi=vbt.RSI.run(data["close"],window=14)
data['RSI']=rsi.rsi
data.tail()

,open,high,low,close,volume,RSI
datetime,,,,,,
2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.92256,40.961362
2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.06052,38.211991
2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.65855,44.645517
2022-01-31 04:30:00,37785.78,38230.00,37778.46,37881.76,1728.81180,49.805624
2022-01-31 05:30:00,37881.75,37951.07,36828.32,36829.30,2907.71096,25.293105


In [129]:
from joblib import load

loaded_clf = load('saved_rf_model.joblib')


In [130]:
from sklearn.preprocessing import StandardScaler
from ta.volatility import AverageTrueRange

data['MA'] = data['close'].rolling(window=window_size_ma).mean()

data['price_diff'] = data['close'].diff()
data['target'] = (data['price_diff'] > threshold_param).astype(int)

data['ATR'] = AverageTrueRange(data['high'], data['low'], data['close'], window=14).average_true_range()

for lag in range(1, 5):
    data[f'lag_{lag}_ATR'] = data['ATR'].shift(lag)

data.dropna(inplace=True)


X_new = data[['MA', 'ATR', 'lag_1_ATR', 'lag_2_ATR', 'lag_3_ATR', 'lag_4_ATR']]

scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new)


predictions = loaded_clf.predict(X_new_scaled)  

data['predictions'] = predictions

In [131]:
price_increase_prob = loaded_clf.predict_proba(X_new_scaled)[:, 1]
price_decrease_prob = loaded_clf.predict_proba(X_new_scaled)[:, 0]

data['entry_long'] = (
    ((data['RSI'] > 20) & (data['RSI'].shift(1) <= 20) & (price_increase_prob >= long_probability_threshold)) |
    ((data['RSI'] > 50) & (data['RSI'].shift(1) <= 50) & (price_increase_prob >= long_probability_threshold)) |
    ((data['RSI'] > 70) & (data['RSI'].shift(1) <= 70) & (price_increase_prob >= long_probability_threshold))
).astype(int)

data['exit_long'] = (
    (((data['RSI'] < 80) & (data['RSI'].shift(1) >= 80)) & (price_decrease_prob >= short_probability_threshold)) |
    (((data['RSI'] < 50) & (data['RSI'].shift(1) >= 50)) & (price_decrease_prob >= short_probability_threshold)) |
    (((data['RSI'] < 20) & (data['RSI'].shift(1) >= 20)) & (price_decrease_prob >= short_probability_threshold))
).astype(int)


In [132]:
num_entry_long = data['entry_long'].sum()
num_exit_long = data['exit_long'].sum()
print(num_entry_long)
print(num_exit_long)

9
545


In [134]:
data['position_size'] = 0.0  

for threshold, size in position_sizes.items():
    condition = (data['RSI'] > threshold) & (data['RSI'].shift(1) <= threshold)
    data.loc[condition, 'position_size'] = size

portfolio = vbt.Portfolio.from_signals(freq="1H",
    close=data['close'], 
    entries=data['entry_long'],  
    exits=data['exit_long'],
    size=data['position_size'],  
    size_type="Percent",
    fees=fees,  
    sl_stop=stop_loss,
    # tp_stop=take_profit,
    sl_trail=sl_trail ,
    init_cash=100000
)

portfolio_stats = portfolio.stats()
portfolio_stats


Start                               2021-02-01 20:30:00
End                                 2022-01-31 05:30:00
Period                                362 days 21:00:00
Start Value                                    100000.0
End Value                                 105179.116804
Total Return [%]                               5.179117
Benchmark Return [%]                          11.064871
Max Gross Exposure [%]                        85.609493
Total Fees Paid                             1189.817282
Max Drawdown [%]                               2.986961
Max Drawdown Duration                 143 days 18:00:00
Total Trades                                          9
Total Closed Trades                                   9
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  66.666667
Best Trade [%]                                 4.939908
Worst Trade [%]                               -3

In [135]:
portfolio.plot().show()